#Imports

In [ ]:
import pandas as pd
import shutil, os, subprocess
import numpy as np
from google.colab import drive
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import pandas as pd
import joblib
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/NIH/')

Mounted at /content/drive


#Dataset reading and pre-processing

In [ ]:
dtype_gmdn = {'PrimaryDI': str}  # Replace 'first_column_name' with the actual column name
dtype_sizes = {'PrimaryDI': str}  # Replace 'first_column_name' with the actual column name

# Read the files into a DataFrame
gmdn_df = pd.read_csv('gmdnTerms.txt', sep='|', dtype=dtype_gmdn)
sizes_df = pd.read_csv('deviceSizes.txt', sep='|', dtype=dtype_sizes)

# Remove rows where sizeType is 'Device Size Text, specify'
sizes_df = sizes_df[sizes_df['sizeType'] != 'Device Size Text, specify']

<ipython-input-2-12868150f357>:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  sizes_df = pd.read_csv('deviceSizes.txt', sep='|', dtype=dtype_sizes)


In [ ]:
# Conversion factors to SI units (meters for length, kilograms for mass, liters for volume, etc.)
conversion_factors = {
    'Millimeter': 1e-3,
    'Centimeter': 1e-2,
    'Micrometer': 1e-6,
    'Milliliter': 1e-3,
    'Gauge': 1,  # Gauge conversion depends on the context
    'French': 1e-3,  # French to meter (for medical devices)
    'Square meter': 1,
    'Inch': 0.0254,
    'Liter': 1,
    'Kilogram': 1,
    'Square centimeter': 1e-4,
    'degree': 1,  # Degrees might need context-specific conversion
    'Gram': 1e-3,
    'Meter': 1,
    'Feet': 0.3048,
    'Pound': 0.453592,
    'Milligram': 1e-6,
    'Nanometer': 1e-9,
    'Fluid Ounce': 0.0295735,
    'Square millimeter': 1e-6,
    'Gallon': 3.78541,
    'Yard': 0.9144,
    'Centiliter': 1e-2,
    'Square inch': 6.4516e-4,
    'Quart': 0.946353,
    'Pint': 0.473176,
    'Microliter': 1e-6,
    'Cubic Inch': 0.0163871,
    'millibar': 100,
    'Pound per Square Inch': 6894.76,
    'Atmosphere': 101325,
    'Femtometer': 1e-15,
    'Decimeter': 0.1,
    'Kilometer': 1e3,
    'Microgram': 1e-9,
    'Deciliter': 1e-1,
    'Hertz': 1,  # Hertz is a frequency unit and may not be converted here
    'KiloPascal': 1e3
}

exception_rows = []

# Function to convert values to their respective SI units
def convert_to_si(row):
    global exception_rows
    unit = row['size (Unit)']

    try:
        value = float(row['size (Value)'])
    except (ValueError, TypeError):
        exception_rows.append(row)
        return np.nan

    if pd.isna(value):
        return np.nan

    # Check if the unit is in the conversion_factors dictionary
    if unit not in conversion_factors:
        # Raise an error if the unit is not in the dictionary
        raise KeyError(f"Unit '{unit}' not found in conversion_factors dictionary")

    factor = conversion_factors[unit]

    if factor is None:
        return np.nan  # or handle the specific case

    return value * factor

In [ ]:
# Apply the conversion to the DataFrame
sizes_df['size (SI Unit)'] = sizes_df.apply(convert_to_si, axis=1)

In [ ]:
# Drop rows where 'size (SI Unit)' is NaN
sizes_df_cleaned = sizes_df.dropna(subset=['size (SI Unit)'])

#Vectorization

In [ ]:
from tqdm import tqdm

# Get unique sizeTypes
unique_size_types = sizes_df_cleaned['sizeType'].unique()

# Initialize the new DataFrame
vector_df = pd.DataFrame(columns=['PrimaryDI'] + list(unique_size_types))
vector_df['PrimaryDI'] = sizes_df_cleaned['PrimaryDI'].unique()
vector_df = vector_df.set_index('PrimaryDI')

# Populate the new DataFrame
for idx, row in tqdm(sizes_df_cleaned.iterrows(), total=len(sizes_df_cleaned), desc="Processing Rows"):
    primary_di = row['PrimaryDI']
    size_type = row['sizeType']
    size_value = row['size (SI Unit)']
    vector_df.loc[primary_di, size_type] = size_value

# Reset the index to make 'PrimaryDI' a normal column
vector_df.reset_index(inplace=True)
vector_df.fillna(0, inplace=True)

Processing Rows: 100%|██████████| 1047125/1047125 [03:30<00:00, 4966.08it/s]


# Inference

**HELPER FUNCTIONS**

In [ ]:
def ensure_folder_exists(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

def remove_file_if_exists(file_path):
    if os.path.exists(file_path):
        os.remove(file_path)

def save_model(model, folder_name, gmdncode):
    model_filename = os.path.join(folder_name, f'{gmdncode}_iso_forest_model.pkl')
    joblib.dump(model, model_filename)

def load_model(folder_name, gmdncode):
    model_filename = os.path.join(folder_name, f'{gmdncode}_iso_forest_model.pkl')
    if os.path.exists(model_filename):
        return joblib.load(model_filename)
    else:
        print(f"Model for gmdncode {gmdncode} not found.")
        return None

def save_anomalies_to_csv(anomalies_filtered, file_path):
    anomalies_filtered.to_csv(
        file_path,
        mode='a',
        header=not os.path.exists(file_path),
        index=False
    )

def process_gmdn_codes(gmdn_df, vector_df, folder_name, file_name, mode):

    # Get unique GMDNCodes
    name_counts = gmdn_df['gmdnCode'].value_counts()

    # Ensure the folder exists
    ensure_folder_exists(folder_name)

    file_path = os.path.join(folder_name, file_name)

    # Delete the previous anomalies file if it exists
    remove_file_if_exists(file_path)

    # Loop over unique gmdnCode values with tqdm progress bar
    for gmdncode in tqdm(dict(name_counts).keys(), desc="Processing gmdnCodes"):
        # Filter gmdn_df to get PrimaryDI values where gmdnCode equals the current gmdncode
        filtered_primary_di = gmdn_df[gmdn_df['gmdnCode'] == gmdncode]['PrimaryDI']

        # Convert to list
        filtered_primary_di_list = filtered_primary_di.tolist()

        # Filter vector_df based on PrimaryDI values obtained from df
        filtered_sizes_df = vector_df[vector_df['PrimaryDI'].isin(filtered_primary_di_list)].copy()

        # Neglecting GMDN Codes with set sizes less than 5
        if filtered_sizes_df.shape[0] < 5:
            continue

        # Extract 'PrimaryDI' and the feature columns
        primary_di = filtered_sizes_df['PrimaryDI']
        features = filtered_sizes_df.drop(columns=['PrimaryDI'])

        # Scale the feature columns
        scaler = StandardScaler()
        features_scaled = scaler.fit_transform(features)

        if mode == 1:
            # Training Mode
            iso_forest = IsolationForest(contamination=0.00001, random_state=42)
            iso_forest.fit(features_scaled)

            # Save the model
            save_model(iso_forest, folder_name, gmdncode)

        elif mode == 2:
            # Inference Mode (load existing model)
            iso_forest = load_model(folder_name, gmdncode)
            if iso_forest is None:
                raise FileNotFoundError(f"Model for gmdnCode {gmdncode} not found.")
                continue

        # Predict anomalies
        anomaly_scores = iso_forest.decision_function(features_scaled)
        anomaly_labels = iso_forest.predict(features_scaled)

        # Add the results to the DataFrame
        filtered_sizes_df.loc[:, 'anomaly_score'] = anomaly_scores
        filtered_sizes_df.loc[:, 'anomaly_label'] = anomaly_labels
        filtered_sizes_df.loc[:, 'gmdnCode'] = str(gmdncode)

        # Filter out potential anomalies (anomaly_label = -1 indicates anomaly)
        anomalies = filtered_sizes_df[filtered_sizes_df['anomaly_label'] == -1].copy()

        # Ensure 'PrimaryDI' and 'gmdnCode' are strings
        anomalies['PrimaryDI'] = anomalies['PrimaryDI'].astype(str)
        anomalies['gmdnCode'] = anomalies['gmdnCode'].astype(str)

        # Select only the desired columns
        anomalies_filtered = anomalies[['PrimaryDI', 'gmdnCode', 'anomaly_score']]

        # Save the anomalies to CSV
        if not anomalies_filtered.empty:
            save_anomalies_to_csv(anomalies_filtered, file_path)

    # Load and return the final anomalies
    if os.path.exists(file_path):
        all_anomalies_df = pd.read_csv(file_path, dtype={'PrimaryDI': str, 'gmdnCode': str})
        return all_anomalies_df
    else:
        print(f"No anomalies were found and saved in {file_path}.")
        return None

**EXAMPLE USAGE**

In [ ]:
"""
Mode descriptions :

1 : Trains models for every GMDN Code and saves the models + Uses these models for inference to output a file containing anomalous PrimaryDIs and the anomaly scores.
(Use this when you're running the script for the first ever time)

2 : Directly loads model from saved pickle files and makes inference for the current data batch
(Use this only after you've run the entire code with MODE = 1 at least once)

"""

# Name of the folder where all the models and the final anomalies csv gets saved
folder_name = 'anomalies_output'

# Name of the anomalies file
anomalies_file_name = 'final_anomalies.csv'

In [ ]:
# To run in mode 1 (train and save models):
final_anomalies_df = process_gmdn_codes(gmdn_df, vector_df, folder_name, anomalies_file_name, mode=1)

# Print the final anomalies DataFrame
if final_anomalies_df is not None:
    print("\n")
    print(final_anomalies_df)

Processing gmdnCodes:   0%|          | 5/12927 [00:04<2:54:51,  1.23it/s]



        PrimaryDI gmdnCode  anomaly_score
0  00850014433642    33961      -0.000005


In [ ]:
# To run in mode 2 (load models and use them for inference):
final_anomalies_df = process_gmdn_codes(gmdn_df, vector_df, folder_name, anomalies_file_name, mode=2)

# Print the final anomalies DataFrame
if final_anomalies_df is not None:
    print("\n")
    print(final_anomalies_df)

Processing gmdnCodes:   0%|          | 5/12927 [00:03<2:16:42,  1.58it/s]



        PrimaryDI gmdnCode  anomaly_score
0  00850014433642    33961      -0.000005
